In [ ]:
import streamlit as st
import leafmap.maplibregl as leafmap
import ibis
from ibis import _
import os
from cng.utils import *
from cng.h3 import *
from cng.maps import *

con = ibis.duckdb.connect(extensions = ["spatial", "h3"])
# install_h3(con)
set_secrets(con)
endpoint = os.environ["AWS_S3_ENDPOINT"]


## Initialize Data ##
pad_pmtiles   = f"https://{endpoint}/public-biodiversity/pad-us-4/pad-us-4.pmtiles"
tract_pmtiles = f"https://{endpoint}/public-social-vulnerability/2022/SVI2022_US_tract.pmtiles"
pad_us    = con.read_parquet(f"https://{endpoint}/public-biodiversity/pad-us-4/pad-us-4.parquet").select("GAP_Sts", "Mang_Type", "row_n", "geom", "GIS_Acres")
tracts_z8 = con.read_parquet(f"https://{endpoint}/public-social-vulnerability/2022-tracts-h3-z8.parquet").mutate(h8 = _.h8.lower())
pad_z8    = con.read_parquet(f"https://{endpoint}/public-biodiversity/pad-us-4/pad-h3-z8.parquet")
mobi      = con.read_parquet(f"https://{endpoint}/public-mobi/hex/all-richness-h8.parquet").select( "h8", "Z").rename(richness = "Z")
svi       = con.read_parquet(f"https://{endpoint}/public-social-vulnerability/2022/SVI2022_US_tract.parquet").select("FIPS", "RPL_THEMES").filter(_.RPL_THEMES > 0)

database = (tracts_z8
  .left_join(mobi, "h8").drop("h8_right")
  .left_join(pad_z8, "h8").drop("h8_right")
  .inner_join(svi, "FIPS")
  .left_join(pad_us, "row_n").drop("row_n_right")
)

gap_codes=["1", "2"]


In [ ]:

# some hexes can be covered by many protected area polygons...
tracts_z8.count().execute()

database.count().execute()

database.group_by(_.h8).agg(n = _.count()).order_by(_.n.desc()).head().execute()

database.filter(_.h8 == "882aac8419fffff").head(50).execute()

In [ ]:
path = "public-data/cache/tmp.csv"
database.filter(_.STATE == "California").drop("geom").to_csv(f"s3://{path}")
url = f"https://{endpoint}/{path}"


In [ ]:


# .to_json() doesn't exist in ibis, use SQL
subset = database.filter(_.STATE == "California").drop("geom").mutate(
        rgb_color=ibis.case()
        .when(_.GAP_Sts == '1', '[38, 51, 61]')
        .when(_.GAP_Sts == '2', '[135, 150, 71]')
        .when(_.GAP_Sts == '3', '[189, 207, 114]')
        .when(_.GAP_Sts == '4', '[109, 110, 109]')
        .else_("[211, 211, 211]")  # or any other default value
        .end()
)
query = ibis.to_sql(subset)
con.raw_sql(f"COPY ({query}) TO 's3://public-data/cache/test.json' (FORMAT JSON, ARRAY true);")



In [ ]:
url = f"https://{endpoint}/public-data/cache/test.json"


In [ ]:
#m = leafmap.Map(style= terrain_style(), center=[-120, 37.6], zoom=4, pitch=55, bearing=20)
m = leafmap.Map(style= "positron", center=[-120, 37.6], zoom=4)
import pydeck as pdk


layer = pdk.Layer("H3HexagonLayer",
            id="data",
            data=url,
            get_hexagon="h8",
            pickable=True,
            stroked=True,
            filled=True,
            extruded=False,
            get_fill_color= "rgb_color",
            line_width_min_pixels=1,
            )

m.add_deck_layers([layer])
m

In [ ]:
subset.head()

In [ ]:

paint = {
        'property': 'GAP_Sts',
        'type': 'categorical',
        'stops': [
            ["1", "#26633d"],
            ["2", "#879647"],
            ["3", "#BBBBBB"],
            ["4", "#F8F8F8"],
        ]
        }
pad_style = {
    "version": 8,
    "sources": {
        "pad": {
            "type": "vector",
            "url": "pmtiles://" + pad_pmtiles,
            "attribution": "US PAD v4"
        },
    },
    "layers": [{
            "id": "public",
            "source": "pad",
            "source-layer": "padus4",
            "type": "fill",
            'filter': ['in', ['get', 'GAP_Sts'], ["literal", gap_codes]],
            "paint": {
                "fill-color": paint,
                "fill-opacity": 0.5
            }
        }]}
m = leafmap.Map(style = "positron")
m.add_pmtiles(pad_pmtiles,  style=pad_style, opacity=0.5, tooltip=True, fit_bounds=True)

m

In [ ]:
import ibis
from ibis import _
con = ibis.duckdb.connect()

(con
 .read_parquet("https://minio.carlboettiger.info/public-biodiversity/pad-us-4/pad-us-4.parquet")
 .filter(_.State_Nm == "CA")
 .group_by(_.GAP_Sts)
 .agg(acres = _.GIS_Acres.sum())
 .order_by(_.GAP_Sts)
 .execute()
)